# PLEASE USE `Cell => Run All` ON THIS NOTEBOOK (MENU OPTION ABOVE).

# THIS NOTEBOOK WILL TAKE 20-30 MINUTES TO COMPLETE SUCCESSFULLY.

# PLEASE BE PATIENT

In [ ]:
import sys

print("Python Version %s" % sys.version)

In [ ]:
!pip list

In [ ]:
%%bash
conda list

# 1. Install the CLI tools

_Note:  Make sure the SageMaker role has access to create EKS Cluster._

In [ ]:
%%bash

source ~/.bash_profile

pip install awscli --upgrade --user

curl --silent --location "https://github.com/weaveworks/eksctl/releases/latest/download/eksctl_$(uname -s)_amd64.tar.gz" | tar xz -C /tmp

sudo mv /tmp/eksctl /usr/local/bin

eksctl version

curl --location -o ./kubectl https://amazon-eks.s3.us-west-2.amazonaws.com/1.18.8/2020-09-18/bin/linux/amd64/kubectl
#curl --location -o ./kubectl https://amazon-eks.s3.us-west-2.amazonaws.com/1.15.10/2020-02-22/bin/linux/amd64/kubectl

chmod +x ./kubectl

sudo mv ./kubectl /usr/local/bin

kubectl version --short --client

curl -o aws-iam-authenticator https://amazon-eks.s3.us-west-2.amazonaws.com/1.18.8/2020-09-18/bin/linux/amd64/aws-iam-authenticator
#curl -o aws-iam-authenticator https://amazon-eks.s3.us-west-2.amazonaws.com/1.15.10/2020-02-22/bin/linux/amd64/aws-iam-authenticator

chmod +x ./aws-iam-authenticator

sudo mv aws-iam-authenticator /usr/local/bin

aws-iam-authenticator version

sudo yum -y install jq gettext

for command in kubectl jq envsubst
  do
    which $command &>/dev/null && echo "$command in path" || echo "$command NOT FOUND"
  done

echo "Completed"

# 2. Configure the Environment Variables

In [ ]:
%%bash

source ~/.bash_profile

export AWS_REGION=$(aws configure get region)
echo "export AWS_REGION=${AWS_REGION}" | tee -a ~/.bash_profile

export AWS_CLUSTER_NAME=workshop
echo "export AWS_CLUSTER_NAME=${AWS_CLUSTER_NAME}" | tee -a ~/.bash_profile

echo "Completed"

# 3. Create the EKS Cluster

# THIS WILL TAKE 20-30 MINUTES.  PLEASE BE PATIENT!

# _If you see `retryable error`s below, this is OK!_

In [ ]:
%%bash

###############################
#
# ONLY RUN THIS CELL ONE TIME!!
#
###############################

source ~/.bash_profile

export AWS_CLUSTER_STATUS=$(aws eks describe-cluster --name ${AWS_CLUSTER_NAME} --region ${AWS_REGION} --query "cluster.status" --output text)

if [ -n "$AWS_CLUSTER_STATUS" ]; then
    if [ $AWS_CLUSTER_STATUS == "ACTIVE" ]; then
        echo "Cluster status:  $AWS_CLUSTER_STATUS."
    else
        echo "Cluster status:  $AWS_CLUSTER_STATUS.  Please wait for status:  ACTIVE"
        exit
    fi
else
    # THIS WILL TAKE 20-30 MINUTES.  PLEASE BE PATIENT. 
    eksctl create cluster \
      --name ${AWS_CLUSTER_NAME} \
      --version 1.18 \
      --nodes 4 \
      --node-type c5.xlarge \
      --node-volume-size 100 \
      --region=${AWS_REGION} \
      --zones=${AWS_REGION}a,${AWS_REGION}b \
      --alb-ingress-access
      
    export AWS_CLUSTER_STATUS=$(aws eks describe-cluster --name ${AWS_CLUSTER_NAME} --region ${AWS_REGION} --query "cluster.status" --output text)
    echo "export AWS_CLUSTER_STATUS=${AWS_CLUSTER_STATUS}" | tee -a ~/.bash_profile
    echo "Cluster status:  $AWS_CLUSTER_STATUS"
fi

# _If you see `retryable error`s  ^^ above ^^, this is OK!_

# AFTER 20-30 MINUTES, YOUR CLUSTER WILL BE CREATED.

In [ ]:
%%bash

source ~/.bash_profile

export AWS_CLUSTER_STATUS=$(aws eks describe-cluster --name ${AWS_CLUSTER_NAME} --region ${AWS_REGION} --query "cluster.status" --output text)

if [ -n "$AWS_CLUSTER_STATUS" ]; then
    if [ $AWS_CLUSTER_STATUS == "ACTIVE" ]; then
        echo "Cluster status:  $AWS_CLUSTER_STATUS.  Please continue."
    else
        echo "Cluster status:  $AWS_CLUSTER_STATUS.  Please wait for status:  ACTIVE"
        exit
    fi
fi